In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install pytorch-lightning
!pip install wandb 
import wandb

#wandb.init(project="CIFAR-100-Batch_reorder_htl", entity="aashlessha")

     |████████████████████████████████| 584 kB 8.3 MB/s 
     |████████████████████████████████| 409 kB 50.5 MB/s 
     |████████████████████████████████| 136 kB 55.6 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 58.3 MB/s 
     |████████████████████████████████| 271 kB 68.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.8 MB 6.9 MB/s 
     |████████████████████████████████| 181 kB 52.8 MB/s 
     |████████████████████████████████| 144 kB 54.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7303aa64e2d9378b561472811d4ff193a91ba5648b9edee33821be64045dc2f8
  Stor

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import torchvision.transforms.functional as TF


import pytorch_lightning as pl
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.functional import accuracy
from torch.optim import Adam
from torch.utils.data import Sampler

import numpy as np
from sklearn.metrics import classification_report
from collections import OrderedDict
from typing import Sized, Iterator

**Defining Surrogate model**

In [4]:
class MobileNet(nn.Module):
    def __init__(self, ch_in, n_classes):
        super(MobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                #nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
                )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                # dw
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                #nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                # pw
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                #nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
                )

        self.model = nn.Sequential(
            conv_bn(ch_in, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, n_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [6]:
model_surrogate = MobileNet(3,100)
model_surrogate.to(device)

MobileNet(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): ReLU(inplace=True)
      (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (3): ReLU(inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (3): ReLU(inplace=True)
    )
    (4): Sequential(
      (0): Con

**Batch Reshuffling**

In [7]:
class BatchShuffleSampler(Sampler[int]):

    def __init__(self, data_source: Sized,  atk_type="lowtohigh", surrogate=model_surrogate, batch_size=32) -> None:
        self.data_source = data_source
        
        self.surrogate = surrogate
        self.batch_size = batch_size
        self.epoch1 = True
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        temp = np.array(list(range(self.batchOrder.shape[0])))
        self.batchOrder = torch.from_numpy(temp)
        print("batch order shape = ", self.batchOrder.shape)
        print("data source length = ", len(data_source))
        print("batch order = ", self.batchOrder[:10])
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)

        self.atk_type = atk_type
    def __getSurrogateloss__(self, batch):
        
        
        with torch.no_grad():
            # random_batch_perm = torch.randperm(batch_size)
            # data_perm = self.data[random_batch_perm]
            # label_perm = self.labels[random_batch_perm]
            
            loss = F.nll_loss(self.surrogate(self.data[batch: batch+1]) 
            ,self.labels[batch: batch+1])

        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            print(self.batchOrder.view(-1, self.batch_size)[:2])
            for i in self.batchOrder.view(-1, self.batch_size):
                yield iter(i)
            self.epoch1 = False
            
        else:
            print('Attacking')
            print("batch order: ",self.batchOrder)

            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])
            print("printing losses now _______")
            print(losses[:5])
            print(torch.argsort(losses)[:5])
            print(losses[torch.argsort(losses)[:5]])
            print("printing losses done _______")
            
            if self.atk_type == "lowtohigh":
              # print("low to high running")
              print(self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size)[:2])
              for i in self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size):
                  yield iter(i)

            elif self.atk_type == "hightolow":
              print("high to low running")
              print(self.batchOrder.shape)
              print(self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size).shape)
              for i in self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size):
                  yield iter(i)

            elif self.atk_type == "oscillating in":
              print("oscillating in running")
              
              asc_sort_losses = torch.argsort(losses).tolist()

              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))

              for i in self.batchOrder[oscillate_inward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            
            elif self.atk_type == "oscillating out":
              print("oscillating out running")
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in self.batchOrder[oscillate_outward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            
            print('Attack successful')
        

    def __len__(self) -> int:
        return self.batchOrder.view(-1, self.batch_size).shape[0]

In [10]:
def Resnet50CIFAR100():
    model = torchvision.models.resnet50(pretrained=False,num_classes=len(trainset.classes))
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()

        self.train_l = train_l
        self.val_l = val_l
        
        self.model = Resnet50CIFAR100()
        
        self.surrogate = surrogate
        self.surrogate_optim = torch.optim.Adam([p for p in self.surrogate.parameters() if p.requires_grad],lr=0.1)
        #self.surrogate_optim = Adam([p for p in self.surrogate.parameters() if p.requires_grad],lr=0.1,momentum=0.3)
        
        

    def forward(self, x):
        out = self.model(x)
        
        return F.log_softmax(out, dim=1)



    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 

        wandb.log({"model loss":loss})
        
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = F.nll_loss(surrogate_logits, y)
        wandb.log({"surrogate loss" : surrogateloss})
        surrogateloss.backward()
        self.surrogate_optim.step()
        # print("loss = ", loss)
        
        return loss
    

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        # self.log('loss', loss.item(), on_step=True)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            wandb.log({f"{stage}_acc" : acc})
            wandb.log({f"{stage}_loss": loss})
            self.log(f"{stage}_loss", loss, prog_bar=True, on_step = True)
            self.log(f"{stage}_acc", acc, prog_bar=True, on_step = True)


    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")


    def configure_optimizers(self):
        return torch.optim.SGD([p for p in self.model.parameters() if p.requires_grad], lr=0.1,momentum=0.3)
        #return Adam([p for p in self.model.parameters() if p.requires_grad], lr=0.1)


    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Batch Reshuffling**
**Low to High**

In [9]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 32

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchShuffleSampler(trainset, atk_type=attack_type[0]))

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = tuple(trainset.classes)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
batch order shape =  torch.Size([49984])
data source length =  50000
batch order =  tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [10]:
from pytorch_lightning.loggers import WandbLogger
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
wandb.init(project="CIFAR-100-Batch_reshuffle_lth", entity="aashlessha")
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=50,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model_source)
torch.save(model_source,'/content/gdrive/MyDrive/Ashlesha/bres_model_cifar100_50_lth.ckpt')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/test
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | ResNet    | 23.7 M
1 | surrogate | MobileNet | 3.3 M 
----------------------------------------
27.0 M    Trainable pa

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([-6.4454e+24, -2.3806e+25, -5.8929e+27, -2.1461e+25,  2.8944e+24])
tensor([ 5950, 40796,  4344, 16242, 25427])
tensor([-1.3193e+28, -1.2610e+28, -1.2490e+28, -1.2348e+28, -1.1496e+28])
printing losses done _______
tensor([[ 5950, 40796,  4344, 16242, 25427, 43575, 46039, 38934, 40607,  3520,
         21290, 21328, 29895, 45782, 45077, 26819, 14444, 25979, 11995, 33983,
         11188, 32128,  5077, 28813,  2936, 42441, 32714, 21210, 23399,  5649,
         46295,  9426],
        [44327, 16942,  1828, 46220, 39404, 43363, 24717, 27576, 26115, 16356,
          5914,  1979, 11264, 43994, 32404, 11841,  1626, 46274, 20889, 40598,
         18799, 30172, 34567, 17114,  8106, 36535, 12775, 15843, 38100, 18746,
         30026,  3007]])
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([-6.4454e+24, -2.3806e+25, -5.8930e+27, -2.1461e+25,  2.8944e+24])
tensor([ 5950, 40796,  4344, 16242, 25427])
tensor([-1.3193e+28, -1.2610e+28, -1.2490e+28, -1.2348e+28, -1.1496e+28])
printing losses done _______
tensor([[ 5950, 40796,  4344, 16242, 25427, 43575, 46039, 38934, 40607,  3520,
         21290, 21328, 29895, 45782, 45077, 26819, 14444, 25979, 11995, 33983,
         11188, 32128,  5077, 28813,  2936, 42441, 32714, 21210, 23399,  5649,
         46295,  9426],
        [44327, 16942,  1828, 46220, 39404, 43363, 24717, 27576, 26115, 16356,
          5914,  1979, 11264, 43994, 32404, 11841,  1626, 46274, 20889, 40598,
         18799, 30172, 34567, 17114,  8106, 36535, 12775, 15843, 38100, 18746,
         30026,  3007]])
Attack successful


Validation: 0it [00:00, ?it/s]

In [11]:
model_source.to(device)
model_source.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model_source(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


               precision    recall  f1-score   support

        apple       0.00      0.00      0.00       100
aquarium_fish       0.04      0.09      0.05       100
         baby       0.00      0.00      0.00       100
         bear       1.00      0.01      0.02       100
       beaver       0.00      0.00      0.00       100
          bed       0.00      0.00      0.00       100
          bee       0.00      0.00      0.00       100
       beetle       0.50      0.05      0.09       100
      bicycle       0.00      0.00      0.00       100
       bottle       0.00      0.00      0.00       100
         bowl       0.33      0.01      0.02       100
          boy       1.00      0.02      0.04       100
       bridge       0.00      0.00      0.00       100
          bus       0.03      0.95      0.06       100
    butterfly       0.00      0.00      0.00       100
        camel       0.75      0.06      0.11       100
          can       0.22      0.02      0.04       100
       ca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Batch Reshuffle HightoLow attack**

In [8]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 32

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchShuffleSampler(trainset, atk_type=attack_type[1]))

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = tuple(trainset.classes)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
batch order shape =  torch.Size([49984])
data source length =  50000
batch order =  tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [11]:
from pytorch_lightning.loggers import WandbLogger
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
wandb.init(project="CIFAR-100-Batch_reshuffle_htl", entity="aashlessha")
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=50,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model_source)
torch.save(model_source,'/content/gdrive/MyDrive/Ashlesha/bres_model_cifar100_50_htl.ckpt')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/test
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | ResNet    | 23.7 M
1 | surrogate | MobileNet | 3.3 M 
----------------------------------------
27.0 M    Trainable pa

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([ 3.3175e+23, -1.5592e+25, -8.4579e+24, -3.0163e+24, -8.3058e+25])
tensor([41971, 16777, 23545, 40723, 39481])
tensor([-2.0610e+28, -1.8576e+28, -1.7934e+28, -1.6713e+28, -1.6650e+28])
printing losses done _______
high to low running
torch.Size([49984])
torch.Size([1562, 32])
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([ 3.3175e+23, -1.5592e+25, -8.4579e+24, -3.0163e+24, -8.3058e+25])
tensor([41971, 16777, 23545, 40723, 39481])
tensor([-2.0610e+28, -1.8576e+28, -1.7934e+28, -1.6713e+28, -1.6650e+28])
printing losses done _______
high to low running
torch.Size([49984])
torch.Size([1562, 32])
Attack successful


Validation: 0it [00:00, ?it/s]

In [12]:
model_source.to(device)
model_source.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model_source(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


               precision    recall  f1-score   support

        apple       0.81      0.13      0.22       100
aquarium_fish       1.00      0.01      0.02       100
         baby       0.20      0.04      0.07       100
         bear       0.00      0.00      0.00       100
       beaver       0.04      0.11      0.06       100
          bed       1.00      0.01      0.02       100
          bee       0.50      0.02      0.04       100
       beetle       0.20      0.01      0.02       100
      bicycle       0.66      0.21      0.32       100
       bottle       1.00      0.03      0.06       100
         bowl       0.17      0.06      0.09       100
          boy       0.50      0.02      0.04       100
       bridge       0.00      0.00      0.00       100
          bus       0.70      0.14      0.23       100
    butterfly       0.00      0.00      0.00       100
        camel       0.80      0.04      0.08       100
          can       1.00      0.02      0.04       100
       ca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Oscillating inwards attack**

In [13]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 32

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchShuffleSampler(trainset, atk_type=attack_type[2]))

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = tuple(trainset.classes)

Files already downloaded and verified
batch order shape =  torch.Size([49984])
data source length =  50000
batch order =  tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [14]:
from pytorch_lightning.loggers import WandbLogger
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
wandb.init(project="CIFAR-100-Batch_reshuffle_osc_in", entity="aashlessha")
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=50,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model_source)
torch.save(model_source,'/content/gdrive/MyDrive/Ashlesha/bres_model_cifar100_50_osc_in.ckpt')

model loss,▅█▅▁▃▃▃▃▃▂▁▂▂▁▁▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
surrogate loss,██████▇██▇██▇█████▃██▆██▇████████▁██▆██▇
val_acc,▅▁▁▂▁▄▁▂▁▁▂▂▁▂▁▄▂▁▂▁▂▅▂▇▅▁▄▂▄▂▅▅▅▂▄█▅▅▄▅
val_loss,▁▃▃▃▅▅▇▆▇▆▆▇▇▆▇▆▇█▅▆▆▅▄▆▄▅▅▅▅▅▄▅▅▇▇▅█▇▇▆
model loss,0.00188
surrogate loss,-1.2820251012903995e+28
val_acc,0.0625
val_loss,17.86781


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | ResNet    | 23.7 M
1 | surrogate | MobileNet | 3.3 M 
----------------------------------------
27.0 M    Trainable params
0         Non-trainable params
27.0 M 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([ 5.5766e+23, -1.3097e+25, -7.1322e+24, -2.9421e+24, -8.0385e+25])
tensor([41971, 16777, 39481, 23545, 32059])
tensor([-2.1123e+28, -1.9402e+28, -1.7937e+28, -1.7784e+28, -1.5876e+28])
printing losses done _______
oscillating in running
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([ 5.5766e+23, -1.3097e+25, -7.1322e+24, -2.9421e+24, -8.0385e+25])
tensor([41971, 16777, 39481, 23545, 32059])
tensor([-2.1123e+28, -1.9402e+28, -1.7937e+28, -1.7784e+28, -1.5876e+28])
printing losses done _______
oscillating in running
Attack successful


Validation: 0it [00:00, ?it/s]

In [15]:
model_source.to(device)
model_source.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model_source(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


               precision    recall  f1-score   support

        apple       0.36      0.67      0.47       100
aquarium_fish       0.52      0.33      0.40       100
         baby       1.00      0.02      0.04       100
         bear       0.50      0.01      0.02       100
       beaver       0.21      0.17      0.19       100
          bed       0.31      0.44      0.36       100
          bee       0.36      0.24      0.29       100
       beetle       0.40      0.17      0.24       100
      bicycle       0.75      0.46      0.57       100
       bottle       0.55      0.46      0.50       100
         bowl       0.21      0.12      0.15       100
          boy       0.05      0.21      0.08       100
       bridge       0.67      0.24      0.35       100
          bus       0.23      0.78      0.35       100
    butterfly       0.31      0.10      0.15       100
        camel       0.50      0.21      0.30       100
          can       0.44      0.29      0.35       100
       ca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Oscillating outwards attack**

In [16]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 32

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchShuffleSampler(trainset, atk_type=attack_type[3]))

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = tuple(trainset.classes)

Files already downloaded and verified
batch order shape =  torch.Size([49984])
data source length =  50000
batch order =  tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [17]:
from pytorch_lightning.loggers import WandbLogger
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
wandb.init(project="CIFAR-100-Batch_reshuffle_osc_out", entity="aashlessha")
model_source = BoilerPlate(trainloader, testloader, model_surrogate)
trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=50,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model_source)
torch.save(model_source,'/content/gdrive/MyDrive/Ashlesha/bres_model_cifar100_50_osc_out.ckpt')

model loss,█▆▅▄▃▄▄▂▃▃▂▂▂▁▂▁▂▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁
surrogate loss,██████████▇██▆█▁█████████▇█▇██▄█████████
val_acc,▃▁▁▃▃▁▄▃▃▃▃▃▅▄▃▅▇▆▆▇▅▃▅▆▃▇▅▅▄▇▆▅▇▇██▆▇▇▅
val_loss,▁▄▄▅▅▅▄▅█▄▆▆▇▆▇▆▅▅▅▃▇▆▆▅▇▄▆▃▇▄▅█▄▄▅▄▆▃▇▇
model loss,0.02732
surrogate loss,-7.264862195344207e+24
val_acc,0.375
val_loss,7.99605


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | ResNet    | 23.7 M
1 | surrogate | MobileNet | 3.3 M 
----------------------------------------
27.0 M    Trainable params
0         Non-trainable params
27.0 M 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([ 6.3454e+23, -1.4137e+25, -9.1336e+24, -2.8569e+24, -5.4168e+25])
tensor([41971, 23545, 16777, 39481, 40723])
tensor([-2.2429e+28, -2.0581e+28, -1.9426e+28, -1.7614e+28, -1.6456e+28])
printing losses done _______
oscillating out running
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([    0,     1,     2,  ..., 49981, 49982, 49983])
printing losses now _______
tensor([ 6.3454e+23, -1.4137e+25, -9.1336e+24, -2.8569e+24, -5.4168e+25])
tensor([41971, 23545, 16777, 39481, 40723])
tensor([-2.2429e+28, -2.0581e+28, -1.9426e+28, -1.7614e+28, -1.6456e+28])
printing losses done _______
oscillating out running
Attack successful


Validation: 0it [00:00, ?it/s]

In [18]:
model_source.to(device)
model_source.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model_source(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


               precision    recall  f1-score   support

        apple       1.00      0.17      0.29       100
aquarium_fish       1.00      0.02      0.04       100
         baby       0.48      0.15      0.23       100
         bear       0.00      0.00      0.00       100
       beaver       0.08      0.08      0.08       100
          bed       0.59      0.10      0.17       100
          bee       0.67      0.02      0.04       100
       beetle       0.78      0.07      0.13       100
      bicycle       1.00      0.14      0.25       100
       bottle       1.00      0.11      0.20       100
         bowl       0.00      0.00      0.00       100
          boy       0.00      0.00      0.00       100
       bridge       0.00      0.00      0.00       100
          bus       0.67      0.06      0.11       100
    butterfly       0.29      0.05      0.09       100
        camel       0.00      0.00      0.00       100
          can       1.00      0.01      0.02       100
       ca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
